In [32]:
import numpy as np
import random
import cProfile

random.seed = 7

In [33]:
states = []
for i in range(10):
    for j in range(2):
        for k in range(10):
            states.append((i, j, k))

In [49]:
pi = {}
for state in states:
    pi[state] = {
        0 : 0,
        1 : 1
    }
for h in range(10):
    for a in range(2):
        for d in range(10):
            if h < 8:
                pi[(h, a, d)][1] = 1
                pi[(h, a, d)][0] = 0
                
            else:
                pi[(h, a, d)][1] = 0
                pi[(h, a, d)][0] = 1
                

In [35]:
# optimal
for h in range(10):
    for a in range(2):
        for d in range(10):
            if a == 1:
                if d in [1, 8, 9] and h < 7:
                    pi[(h, a, d)][1] = 1
                    pi[(h, a, d)][0] = 0
                
                elif h < 6:
                    pi[(h, a, d)][1] = 1
                    pi[(h, a, d)][0] = 0
                
                else:
                    pi[(h, a, d)][1] = 0
                    pi[(h, a, d)][0] = 1
            
            else:
                if h < 5 and d in [0, 6, 7, 8, 9]:
                    pi[(h, a, d)][1] = 1
                    pi[(h, a, d)][0] = 0
                
                elif h == 0 and d in [1, 2]:
                    pi[(h, a, d)][1] = 1
                    pi[(h, a, d)][0] = 0
                
                else:
                    pi[(h, a, d)][1] = 0
                    pi[(h, a, d)][0] = 1
                     

In [36]:
# random policy

for h in range(10):
    for a in range(2):
        for d in range(10):
            r = np.random.random()
            pi[(h, a, d)][1] = r
            pi[(h, a, d)][0] = 1 - r

In [37]:
pi = {(0, 0, 0): {0: 0.3166552189865136, 1: 0.6833447810134864},
 (0, 0, 1): {0: 0.3159130608646631, 1: 0.6840869391353369},
 (0, 0, 2): {0: 0.8677363366814479, 1: 0.1322636633185521},
 (0, 0, 3): {0: 0.32620387235744974, 1: 0.6737961276425503},
 (0, 0, 4): {0: 0.6470092412901098, 1: 0.35299075870989016},
 (0, 0, 5): {0: 0.1580346287065818, 1: 0.8419653712934182},
 (0, 0, 6): {0: 0.8729254792603178, 1: 0.12707452073968217},
 (0, 0, 7): {0: 0.4089136021409808, 1: 0.5910863978590192},
 (0, 0, 8): {0: 0.906771643324633, 1: 0.093228356675367},
 (0, 0, 9): {0: 0.568851586104125, 1: 0.431148413895875},
 (0, 1, 0): {0: 0.608497890788322, 1: 0.39150210921167805},
 (0, 1, 1): {0: 0.7911428751158931, 1: 0.2088571248841069},
 (0, 1, 2): {0: 0.6022918319594668, 1: 0.3977081680405332},
 (0, 1, 3): {0: 0.3873199738728047, 1: 0.6126800261271953},
 (0, 1, 4): {0: 0.6981632848174483, 1: 0.30183671518255173},
 (0, 1, 5): {0: 0.28995835741913234, 1: 0.7100416425808677},
 (0, 1, 6): {0: 0.5715821227757072, 1: 0.4284178772242928},
 (0, 1, 7): {0: 0.6426151049684717, 1: 0.35738489503152826},
 (0, 1, 8): {0: 0.4459803499281295, 1: 0.5540196500718705},
 (0, 1, 9): {0: 0.3231427702956938, 1: 0.6768572297043062},
 (1, 0, 0): {0: 0.09019772608358112, 1: 0.9098022739164189},
 (1, 0, 1): {0: 0.8429580916462724, 1: 0.15704190835372756},
 (1, 0, 2): {0: 0.9955344528432324, 1: 0.004465547156767613},
 (1, 0, 3): {0: 0.8643442727172866, 1: 0.13565572728271336},
 (1, 0, 4): {0: 0.012550687956674578, 1: 0.9874493120433254},
 (1, 0, 5): {0: 0.04800174820884917, 1: 0.9519982517911508},
 (1, 0, 6): {0: 0.8532380854384005, 1: 0.14676191456159948},
 (1, 0, 7): {0: 0.15376053160813907, 1: 0.8462394683918609},
 (1, 0, 8): {0: 0.2392944353313201, 1: 0.7607055646686799},
 (1, 0, 9): {0: 0.516848268050141, 1: 0.48315173194985905},
 (1, 1, 0): {0: 0.015967393637830063, 1: 0.9840326063621699},
 (1, 1, 1): {0: 0.8866199336406242, 1: 0.11338006635937581},
 (1, 1, 2): {0: 0.9414740674771973, 1: 0.05852593252280269},
 (1, 1, 3): {0: 0.608254680487029, 1: 0.391745319512971},
 (1, 1, 4): {0: 0.9062023477946715, 1: 0.09379765220532854},
 (1, 1, 5): {0: 0.031167299888746114, 1: 0.9688327001112539},
 (1, 1, 6): {0: 0.6901257181392616, 1: 0.3098742818607384},
 (1, 1, 7): {0: 0.9246287902058445, 1: 0.07537120979415546},
 (1, 1, 8): {0: 0.23827348092820266, 1: 0.7617265190717973},
 (1, 1, 9): {0: 0.47513469545084397, 1: 0.524865304549156},
 (2, 0, 0): {0: 0.20017886057730594, 1: 0.7998211394226941},
 (2, 0, 1): {0: 0.485304238836722, 1: 0.514695761163278},
 (2, 0, 2): {0: 0.9271126581906856, 1: 0.07288734180931444},
 (2, 0, 3): {0: 0.4760606771499025, 1: 0.5239393228500975},
 (2, 0, 4): {0: 0.28049673215641746, 1: 0.7195032678435825},
 (2, 0, 5): {0: 0.6848536761641474, 1: 0.3151463238358526},
 (2, 0, 6): {0: 0.346997417044147, 1: 0.653002582955853},
 (2, 0, 7): {0: 0.6251861768309102, 1: 0.37481382316908984},
 (2, 0, 8): {0: 0.03667369336552129, 1: 0.9633263066344787},
 (2, 0, 9): {0: 0.7974655161353155, 1: 0.20253448386468453},
 (2, 1, 0): {0: 0.09531600127650863, 1: 0.9046839987234914},
 (2, 1, 1): {0: 0.5944081654853413, 1: 0.40559183451465874},
 (2, 1, 2): {0: 0.40298243487784924, 1: 0.5970175651221508},
 (2, 1, 3): {0: 0.1997528518448497, 1: 0.8002471481551503},
 (2, 1, 4): {0: 0.6314588584744929, 1: 0.3685411415255071},
 (2, 1, 5): {0: 0.9906784572703347, 1: 0.009321542729665344},
 (2, 1, 6): {0: 0.39239802641734733, 1: 0.6076019735826527},
 (2, 1, 7): {0: 0.16671657353631986, 1: 0.8332834264636801},
 (2, 1, 8): {0: 0.8646589190287444, 1: 0.13534108097125563},
 (2, 1, 9): {0: 0.10315038601073612, 1: 0.8968496139892639},
 (3, 0, 0): {0: 0.9867042142983948, 1: 0.013295785701605167},
 (3, 0, 1): {0: 0.5082501018180302, 1: 0.4917498981819698},
 (3, 0, 2): {0: 0.8161509843765363, 1: 0.18384901562346367},
 (3, 0, 3): {0: 0.11411336362994284, 1: 0.8858866363700572},
 (3, 0, 4): {0: 0.9405908701151797, 1: 0.05940912988482028},
 (3, 0, 5): {0: 0.7698076488951155, 1: 0.23019235110488445},
 (3, 0, 6): {0: 0.0015164514539435237, 1: 0.9984835485460565},
 (3, 0, 7): {0: 0.7070412935456253, 1: 0.2929587064543747},
 (3, 0, 8): {0: 0.5494506759195567, 1: 0.45054932408044335},
 (3, 0, 9): {0: 0.6642785615269252, 1: 0.3357214384730748},
 (3, 1, 0): {0: 0.946453568987306, 1: 0.05354643101269396},
 (3, 1, 1): {0: 0.6094617334471741, 1: 0.39053826655282586},
 (3, 1, 2): {0: 0.6036674483552378, 1: 0.3963325516447622},
 (3, 1, 3): {0: 0.5517358743019304, 1: 0.4482641256980696},
 (3, 1, 4): {0: 0.8449940615497589, 1: 0.15500593845024113},
 (3, 1, 5): {0: 0.7473385153355379, 1: 0.25266148466446214},
 (3, 1, 6): {0: 0.05505980737648364, 1: 0.9449401926235164},
 (3, 1, 7): {0: 0.8840011547142552, 1: 0.11599884528574478},
 (3, 1, 8): {0: 0.4844570132814856, 1: 0.5155429867185144},
 (3, 1, 9): {0: 0.9724649196439212, 1: 0.027535080356078767},
 (4, 0, 0): {0: 0.9076676314946432, 1: 0.09233236850535675},
 (4, 0, 1): {0: 0.7020911888487535, 1: 0.29790881115124646},
 (4, 0, 2): {0: 0.6565556942745354, 1: 0.3434443057254646},
 (4, 0, 3): {0: 0.2864306168181825, 1: 0.7135693831818175},
 (4, 0, 4): {0: 0.055936167624760835, 1: 0.9440638323752392},
 (4, 0, 5): {0: 0.11647796252391396, 1: 0.883522037476086},
 (4, 0, 6): {0: 0.8315856859875979, 1: 0.16841431401240214},
 (4, 0, 7): {0: 0.8498404891391668, 1: 0.15015951086083323},
 (4, 0, 8): {0: 0.36379899804465154, 1: 0.6362010019553485},
 (4, 0, 9): {0: 0.2383116009527637, 1: 0.7616883990472363},
 (4, 1, 0): {0: 0.5865283703542767, 1: 0.41347162964572326},
 (4, 1, 1): {0: 0.9927318282528296, 1: 0.007268171747170427},
 (4, 1, 2): {0: 0.6425726450761743, 1: 0.3574273549238257},
 (4, 1, 3): {0: 0.9583197118494834, 1: 0.04168028815051661},
 (4, 1, 4): {0: 0.26620645318747926, 1: 0.7337935468125207},
 (4, 1, 5): {0: 0.29929983341169186, 1: 0.7007001665883081},
 (4, 1, 6): {0: 0.18631131679410462, 1: 0.8136886832058954},
 (4, 1, 7): {0: 0.6833555231393892, 1: 0.3166444768606108},
 (4, 1, 8): {0: 0.6037639051165429, 1: 0.3962360948834571},
 (4, 1, 9): {0: 0.8633672541856416, 1: 0.13663274581435836},
 (5, 0, 0): {0: 0.26475762132810476, 1: 0.7352423786718952},
 (5, 0, 1): {0: 0.6775146995507147, 1: 0.32248530044928525},
 (5, 0, 2): {0: 0.5074099063304959, 1: 0.4925900936695041},
 (5, 0, 3): {0: 0.5799378772984718, 1: 0.4200621227015282},
 (5, 0, 4): {0: 0.22423608169431108, 1: 0.7757639183056889},
 (5, 0, 5): {0: 0.8834021869603098, 1: 0.11659781303969019},
 (5, 0, 6): {0: 0.3327896832676819, 1: 0.6672103167323181},
 (5, 0, 7): {0: 0.4088345925578756, 1: 0.5911654074421244},
 (5, 0, 8): {0: 0.8361908767613481, 1: 0.16380912323865193},
 (5, 0, 9): {0: 0.08435336596247522, 1: 0.9156466340375248},
 (5, 1, 0): {0: 0.962464378660602, 1: 0.03753562133939803},
 (5, 1, 1): {0: 0.12366945186781164, 1: 0.8763305481321884},
 (5, 1, 2): {0: 0.8823904807662744, 1: 0.11760951923372565},
 (5, 1, 3): {0: 0.9074655245661863, 1: 0.09253447543381366},
 (5, 1, 4): {0: 0.7033849411677874, 1: 0.2966150588322126},
 (5, 1, 5): {0: 0.4518637856619928, 1: 0.5481362143380072},
 (5, 1, 6): {0: 0.7246275213720678, 1: 0.27537247862793224},
 (5, 1, 7): {0: 0.32939470911353963, 1: 0.6706052908864604},
 (5, 1, 8): {0: 0.8723931272101625, 1: 0.1276068727898375},
 (5, 1, 9): {0: 0.3622413653174642, 1: 0.6377586346825358},
 (6, 0, 0): {0: 0.773433482475909, 1: 0.22656651752409096},
 (6, 0, 1): {0: 0.6426299777183488, 1: 0.35737002228165116},
 (6, 0, 2): {0: 0.3842705903400251, 1: 0.6157294096599749},
 (6, 0, 3): {0: 0.24471406660325112, 1: 0.7552859333967489},
 (6, 0, 4): {0: 0.11766689168273226, 1: 0.8823331083172677},
 (6, 0, 5): {0: 0.46849251164977346, 1: 0.5315074883502265},
 (6, 0, 6): {0: 0.6481666543525213, 1: 0.3518333456474787},
 (6, 0, 7): {0: 0.09660162990498267, 1: 0.9033983700950173},
 (6, 0, 8): {0: 0.3533509828837812, 1: 0.6466490171162188},
 (6, 0, 9): {0: 0.3581175677991818, 1: 0.6418824322008182},
 (6, 1, 0): {0: 0.4870765441332763, 1: 0.5129234558667237},
 (6, 1, 1): {0: 0.16711091167458625, 1: 0.8328890883254138},
 (6, 1, 2): {0: 0.3543551601941626, 1: 0.6456448398058374},
 (6, 1, 3): {0: 0.9780247350011, 1: 0.021975264998900013},
 (6, 1, 4): {0: 0.45741739915054636, 1: 0.5425826008494536},
 (6, 1, 5): {0: 0.2825932687409939, 1: 0.7174067312590061},
 (6, 1, 6): {0: 0.28969203399313725, 1: 0.7103079660068627},
 (6, 1, 7): {0: 0.44014712019318813, 1: 0.5598528798068119},
 (6, 1, 8): {0: 0.7175989616282273, 1: 0.28240103837177266},
 (6, 1, 9): {0: 0.9593022103884924, 1: 0.04069778961150761},
 (7, 0, 0): {0: 0.762206894781154, 1: 0.237793105218846},
 (7, 0, 1): {0: 0.38835320088251013, 1: 0.6116467991174899},
 (7, 0, 2): {0: 0.9556532444456954, 1: 0.04434675555430456},
 (7, 0, 3): {0: 0.7865542268864749, 1: 0.21344577311352508},
 (7, 0, 4): {0: 0.7644756121947767, 1: 0.23552438780522333},
 (7, 0, 5): {0: 0.7141793125193105, 1: 0.28582068748068945},
 (7, 0, 6): {0: 0.7545610785268784, 1: 0.24543892147312163},
 (7, 0, 7): {0: 0.18938962489462075, 1: 0.8106103751053793},
 (7, 0, 8): {0: 0.643798277055767, 1: 0.356201722944233},
 (7, 0, 9): {0: 0.37173736299446947, 1: 0.6282626370055305},
 (7, 1, 0): {0: 0.3569124916699965, 1: 0.6430875083300035},
 (7, 1, 1): {0: 0.6206273215153476, 1: 0.3793726784846524},
 (7, 1, 2): {0: 0.5606389701725585, 1: 0.4393610298274415},
 (7, 1, 3): {0: 0.3276492968394814, 1: 0.6723507031605186},
 (7, 1, 4): {0: 0.1691292076530635, 1: 0.8308707923469365},
 (7, 1, 5): {0: 0.7307754528844332, 1: 0.26922454711556676},
 (7, 1, 6): {0: 0.1471737217237954, 1: 0.8528262782762046},
 (7, 1, 7): {0: 0.0940700825733718, 1: 0.9059299174266282},
 (7, 1, 8): {0: 0.8989171574463923, 1: 0.10108284255360767},
 (7, 1, 9): {0: 0.6636975786650926, 1: 0.3363024213349074},
 (8, 0, 0): {0: 0.2622452186655052, 1: 0.7377547813344948},
 (8, 0, 1): {0: 0.8236789343633787, 1: 0.1763210656366213},
 (8, 0, 2): {0: 0.16942412697776488, 1: 0.8305758730222351},
 (8, 0, 3): {0: 0.7297043689471422, 1: 0.2702956310528578},
 (8, 0, 4): {0: 0.8228476695569763, 1: 0.1771523304430237},
 (8, 0, 5): {0: 0.23927402884392457, 1: 0.7607259711560754},
 (8, 0, 6): {0: 0.5041508056579262, 1: 0.49584919434207375},
 (8, 0, 7): {0: 0.054966911655535466, 1: 0.9450330883444645},
 (8, 0, 8): {0: 0.9138356917919186, 1: 0.08616430820808141},
 (8, 0, 9): {0: 0.9879136090993453, 1: 0.012086390900654664},
 (8, 1, 0): {0: 0.9367510163647013, 1: 0.06324898363529874},
 (8, 1, 1): {0: 0.49415337400654025, 1: 0.5058466259934598},
 (8, 1, 2): {0: 0.9766880791707124, 1: 0.02331192082928757},
 (8, 1, 3): {0: 0.7827234596028554, 1: 0.2172765403971446},
 (8, 1, 4): {0: 0.14567722509244385, 1: 0.8543227749075561},
 (8, 1, 5): {0: 0.5643867890903692, 1: 0.4356132109096308},
 (8, 1, 6): {0: 0.8589840532293486, 1: 0.14101594677065143},
 (8, 1, 7): {0: 0.1278562080734813, 1: 0.8721437919265187},
 (8, 1, 8): {0: 0.8248585372326371, 1: 0.17514146276736287},
 (8, 1, 9): {0: 0.2323954171870929, 1: 0.7676045828129071},
 (9, 0, 0): {0: 0.7966185290151464, 1: 0.2033814709848536},
 (9, 0, 1): {0: 0.7665827399894394, 1: 0.23341726001056062},
 (9, 0, 2): {0: 0.2805993032549602, 1: 0.7194006967450398},
 (9, 0, 3): {0: 0.6729199860958596, 1: 0.32708001390414043},
 (9, 0, 4): {0: 0.7091461543464477, 1: 0.2908538456535523},
 (9, 0, 5): {0: 0.1902890794124077, 1: 0.8097109205875923},
 (9, 0, 6): {0: 0.37194434409153887, 1: 0.6280556559084611},
 (9, 0, 7): {0: 0.8698749722790873, 1: 0.13012502772091272},
 (9, 0, 8): {0: 0.6283929654746186, 1: 0.37160703452538135},
 (9, 0, 9): {0: 0.9454495488133771, 1: 0.05455045118662294},
 (9, 1, 0): {0: 0.2322845585423443, 1: 0.7677154414576557},
 (9, 1, 1): {0: 0.03716839527625404, 1: 0.962831604723746},
 (9, 1, 2): {0: 0.5764294422381442, 1: 0.4235705577618558},
 (9, 1, 3): {0: 0.23094254289514848, 1: 0.7690574571048515},
 (9, 1, 4): {0: 0.3430553720931492, 1: 0.6569446279068508},
 (9, 1, 5): {0: 0.1472299921418625, 1: 0.8527700078581375},
 (9, 1, 6): {0: 0.5046772030199045, 1: 0.4953227969800955},
 (9, 1, 7): {0: 0.5817406785294752, 1: 0.4182593214705248},
 (9, 1, 8): {0: 0.5637885628600947, 1: 0.4362114371399053},
 (9, 1, 9): {0: 0.6934795323177168, 1: 0.30652046768228325}}


# Value Function
3 dimensional array, where first index is \[cardsum(12 to 21) - 12\] (_for indexing_); second index is 0 if no usable ace and 1 if any; third index is dealer card \[(1 to 10)-1\](_for indexing_).

In [38]:
V = np.zeros((10, 2, 10))
visit_count = np.zeros((10, 2, 10))

# BlackJack Monte-Carlo Strategy

In [39]:
def update_values(states_visited, r, V, visit_count):
    for h, a, d in states_visited:
        V[h][a][d] = V[h][a][d] + (1/visit_count[h][a][d])*(r - V[h][a][d])

In [40]:
def blackjack_hand_result(V, visit_count):
    #Deck of cards, 4 types of 10
    
    deck=np.array([1,2,3,4,5,6,7,8,9,10,10,10,10])
    
    #Creating a random hand if none is input
    
    player_hand=[random.choice(deck), random.choice(deck)]
        
    dealer_card=random.choice(deck)
    
    # keeps track of all states that the hand has been in during one play
    states_visited = []
    
    player_sum = sum(player_hand)
    
    # an ace that can be used as an 11 i.e. the sum of the cards is less than 12
    usable_ace = 0
    if 1 in player_hand:
        usable_ace = 1
        player_sum += 10
    
    # update states
    if player_sum > 11:
        state = (player_sum-12, usable_ace, dealer_card-1)
        states_visited.append(state)
        visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
        
    # keeps track of dealers hand
    dealer_cards=[dealer_card]
    

    # Seeing if player hit blackjack
    if player_sum == 21:

        #Seeing if casino also hit blackjack, in which case tie

        dealer_cards.append(random.choice(deck))
        if 1 in (dealer_cards) and sum(dealer_cards)==11:
            update_values(states_visited, 0, V, visit_count)
            return 0
        
        update_values(states_visited, 1, V, visit_count)
        return 1
    

    else:
    # Seeing how often it says to 'hit'
        hit = 1
        stay = 0
        while player_sum < 12 or np.random.random() < pi[state][hit]:
            #Adding one card for every hit
            new_card = random.choice(deck)
            player_hand.append(new_card)
            player_sum = sum(player_hand)
            
            usable_ace = 0
            
            if 1 in player_hand and player_sum < 12:
                usable_ace = 1
                player_sum += 10
            
            state = (player_sum-12, usable_ace, dealer_card-1)
            
            #Player loses bet if hand goes above 21
            if player_sum > 21:
                update_values(states_visited, -1, V, visit_count)
                return -1
            
            if player_sum > 11:
                states_visited.append(state)
                visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
            
    while True:
        #Plays out the blackjack hand from dealer's side

        #Give dealer extra card if loop hasn't broken
        dealer_cards.append(random.choice(deck))


        #Keep track of sum of dealer's cards
        dealer_score= sum(dealer_cards)

        #Keep track of soft score if dealer has an ace

        soft_score= dealer_score
        if dealer_score<=11 and 1 in dealer_cards:
            soft_score+=10

        #If dealer gets blackjack you lose even if you have 21
        if len(dealer_cards)==2 and soft_score==21:
            update_values(states_visited, -1, V, visit_count)
            return -1
        
        #Dealer stays on all 17s
        if soft_score>=17:

            #If dealer bust, player wins bet
            if soft_score>21:
                update_values(states_visited, 1, V, visit_count)
                return 1

            #If player has more than dealer, player wins bet
            if player_sum>soft_score:
                update_values(states_visited, 1, V, visit_count)
                return 1

            #Tie means no money changes hands
            if player_sum==soft_score:
                update_values(states_visited, 0, V, visit_count)
                return 0

            #If player has lower, player loses bet
            if player_sum<soft_score:
                update_values(states_visited, -1, V, visit_count)
                return -1


# Multiple Hand Sim, Test to see house edge in Blackjack

In [41]:
def blackjack_sim(n_hands, V, visit_count):
    for i in range(n_hands):
        blackjack_hand_result(V, visit_count)
     

In [50]:
V = np.zeros((10, 2, 10))
visit_count = np.zeros((10, 2, 10))
cProfile.run("blackjack_sim(200_000, V, visit_count)")

7694246 function calls in 6.725 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   200000    1.012    0.000    1.012    0.000 <ipython-input-39-d6350201368c>:1(update_values)
   200000    2.564    0.000    6.577    0.000 <ipython-input-40-3c797b5005c0>:1(blackjack_hand_result)
        1    0.147    0.147    6.725    6.725 <ipython-input-41-f3b2d7b599ea>:1(blackjack_sim)
        1    0.000    0.000    6.725    6.725 <string>:1(<module>)
  1018519    0.735    0.000    1.060    0.000 random.py:224(_randbelow)
  1018519    0.719    0.000    1.880    0.000 random.py:256(choice)
        1    0.000    0.000    6.725    6.725 {built-in method builtins.exec}
  1143866    0.116    0.000    0.116    0.000 {built-in method builtins.len}
   610473    0.341    0.000    0.341    0.000 {built-in method builtins.sum}
   200000    0.417    0.000    0.417    0.000 {built-in method numpy.core.multiarray.array}
   729990    0.099    0.000    0.

In [51]:
import plotly.graph_objects as go

y_values = []
x_values = []

for i in range(12, 22):
    for k in range(12, 22):
        x_values.append(i)

for i in range(10):
    for k in range(1, 11):
        y_values.append(k)



## Graph for hands without a usable ace

In [52]:
without_ace = np.array([[0.]*10]*10)
for i in range(len(V)):
    for j in range(len(V[0][0])):
        without_ace[i][j] = V[i][0][j]
        
z_values = np.array(without_ace).flatten()

without_ace_fig = go.Figure(data=[go.Mesh3d(x=np.array(x_values),
                   y=(np.array(y_values)),
                   z=(np.array(z_values)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

without_ace_fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=10, range=[12, 21],),
                     yaxis = dict(nticks=10, range=[1, 10],),
                     zaxis = dict(nticks=5, range=[-1, 1],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

without_ace_fig.show()

In [53]:
pi

{(0, 0, 0): {0: 0, 1: 1},
 (0, 0, 1): {0: 0, 1: 1},
 (0, 0, 2): {0: 0, 1: 1},
 (0, 0, 3): {0: 0, 1: 1},
 (0, 0, 4): {0: 0, 1: 1},
 (0, 0, 5): {0: 0, 1: 1},
 (0, 0, 6): {0: 0, 1: 1},
 (0, 0, 7): {0: 0, 1: 1},
 (0, 0, 8): {0: 0, 1: 1},
 (0, 0, 9): {0: 0, 1: 1},
 (0, 1, 0): {0: 0, 1: 1},
 (0, 1, 1): {0: 0, 1: 1},
 (0, 1, 2): {0: 0, 1: 1},
 (0, 1, 3): {0: 0, 1: 1},
 (0, 1, 4): {0: 0, 1: 1},
 (0, 1, 5): {0: 0, 1: 1},
 (0, 1, 6): {0: 0, 1: 1},
 (0, 1, 7): {0: 0, 1: 1},
 (0, 1, 8): {0: 0, 1: 1},
 (0, 1, 9): {0: 0, 1: 1},
 (1, 0, 0): {0: 0, 1: 1},
 (1, 0, 1): {0: 0, 1: 1},
 (1, 0, 2): {0: 0, 1: 1},
 (1, 0, 3): {0: 0, 1: 1},
 (1, 0, 4): {0: 0, 1: 1},
 (1, 0, 5): {0: 0, 1: 1},
 (1, 0, 6): {0: 0, 1: 1},
 (1, 0, 7): {0: 0, 1: 1},
 (1, 0, 8): {0: 0, 1: 1},
 (1, 0, 9): {0: 0, 1: 1},
 (1, 1, 0): {0: 0, 1: 1},
 (1, 1, 1): {0: 0, 1: 1},
 (1, 1, 2): {0: 0, 1: 1},
 (1, 1, 3): {0: 0, 1: 1},
 (1, 1, 4): {0: 0, 1: 1},
 (1, 1, 5): {0: 0, 1: 1},
 (1, 1, 6): {0: 0, 1: 1},
 (1, 1, 7): {0: 0, 1: 1},
 (1, 1, 8): 

## Graph for Hands with a usable ace

In [54]:
with_ace = np.array([[0.]*10]*10)
for i in range(len(V)):
    for j in range(len(V[0][0])):
        with_ace[i][j] = V[i][1][j]
        
z_values = np.array(with_ace).flatten()

with_ace_fig = go.Figure(data=[go.Mesh3d(x=np.array(x_values),
                   y=(np.array(y_values)),
                   z=(np.array(z_values)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

with_ace_fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=10, range=[12, 21],),
                     yaxis = dict(nticks=10, range=[1, 10],),
                     zaxis = dict(nticks=5, range=[-1, 1],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

with_ace_fig.show()